In [1]:
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import os
import tensorflow as tf

Using TensorFlow backend.


In [2]:
"""Load Train Data"""
#code for reading data
imgdir = []   #shall contain the full path of the image
ytr = []         #shall contain label of each image
#xtr = np.matrix([[]])         #shall contain each image

#xtr = np.zeros((257,262144))   #modified
xtr = np.zeros((1300,10000))

datatype = ['train', 'valid']
studylabel = {'negative':0, 'positive':1}

BASE_DIR = 'mura_humerus'

traindir = BASE_DIR + '/' + datatype[0]  #shows training directory

patients = os.listdir(traindir)  #all the patient ID's are now into this

i = 0
for patient in patients:
    directory = traindir + '/' + patient    #patient directory train/patientxxxx/
    studies = os.listdir(directory)         #creates list of studies in each patient directory
    for study in studies:
        images = os.listdir(directory + '/' + study)
        for image in images:
            imgdir = np.append(imgdir, directory + '/' + study + '/' + image)    #add image addresses in array
            ytr = np.append(ytr, studylabel[study.split('_')[1]])        #keep adding labels of each images
            file = mpimg.imread(imgdir[i])
            if (len(np.shape(file))>2):
                pixels = np.matrix(file[:,:,0])     #takes matrix format of one image
                pixels = np.reshape(pixels, (1,len(pixels)*len(pixels.T)))   #flattens the image to 1-D
            else:
                pixels = np.matrix(file)
                pixels = np.reshape(pixels, (1,len(pixels)*len(pixels.T)))
                
            if (np.shape(pixels)[1] < 10000):    #modified from 262144
                zeroarray = np.matrix(np.zeros((1,(10000-np.shape(pixels)[1]))))
                pixels = np.concatenate((pixels,zeroarray), axis=1)
                #print(pixels)
                #xtr = np.append(xtr, np.array(file[:,:,0]), axis=0)
                ###xtr = np.concatenate(xtr,pixels, axis=1)               #load and add images in 
                xtr[i] = pixels               #load and add images in 
                #print ('i from if=', i)
            else:
                xtr[i] = np.matrix(pixels[:,0:10000])     #modified from 262144
                #print(pixels)
                #print ('i from else=', i)
                
            i=i+1
            
xtrain = xtr[0:i,:]
ytrain = ytr

In [3]:
"""Normalize data"""
for i in range (len(xtrain)):
    if np.amax(xtrain[i]) != 0:
        xtrain[i] = xtrain[i] / np.amax(xtrain[i])

In [4]:
"""Eliminating all zero data"""
xtr = np.zeros((np.shape(xtrain)))
ytr = []
j = 0
i = 0
while i < (len(ytrain)):
    if np.amax(xtrain[i]) != 0:
        xtr[j] = xtrain[i]
        ytr.append(ytrain[i])
        j = j+1
    i = i+1
    
xtr = xtr[0:j,:]

In [5]:
"""Verification"""
print(np.shape(xtrain))
print(np.shape(xtr))
print(np.shape(ytrain))
print(np.shape(ytr))

(1272, 10000)
(1260, 10000)
(1272,)
(1260,)


In [6]:
"""Function for calculating error"""         #approved
def errcalc(y,ypred):
    if len(y) != len(ypred):
        print('The inputs must be same size')
        return -1
    else:
        error = np.sum(np.absolute(np.subtract(y,ypred)))
    return error/(len(y))

In [7]:
"""Function for creating a 5 layer NN. Creates, Compiles and Returns the Model"""   #approved
def make_nn(inlayersize, h1size, h2size, h3size, h4size, h5size, h6size, h7size, h8size, h9size, h10size, outlayersize):
    #Training data in 'xtr'
    #Labels in 'ytr'

    #input layer
    inp_layer = Input(shape=(inlayersize,))


    #hidden layer 1
    H1 = Dense(h1size, activation = 'relu')(inp_layer)

    #hidden layer 2
    H2 = Dense(h2size, activation='relu')(H1)

    #hidden layer 3
    H3 = Dense(h3size, activation = 'relu')(H2)

    #hidden layer 4
    H4 = Dense(h4size, activation = 'relu')(H3)
    
    #hidden layer 5
    H5 = Dense(h5size, activation = 'relu')(H4)

    #hidden layer 6
    H6 = Dense(h6size, activation = 'relu')(H5)

    #hidden layer 7
    H7 = Dense(h7size, activation = 'relu')(H6)
    
    #hidden layer 8
    H8 = Dense(h8size, activation = 'relu')(H7)
    
    #hidden layer 9
    H9 = Dense(h9size, activation = 'relu')(H8)

    #hidden layer 10
    H10 = Dense(h10size, activation = 'relu')(H9)

    #output layer
    out_layer = Dense(outlayersize, activation='sigmoid')(H10)



    ###creating the neural network
    shallow_nn = Model(inp_layer, out_layer)


    ###Compiling model
    shallow_nn.compile(optimizer='adadelta', loss='binary_crossentropy')

    ### Returning compiled model
    
    return shallow_nn

In [8]:
"""Function for fitting into a model"""        #approved
def train_nn(shallow_nn,xtr,ytr, batchsize, epchs, shffle):
    shallow_nn.fit(xtr, ytr, batch_size=batchsize, epochs=epchs, shuffle=shffle)
    return shallow_nn

In [9]:
"""Function for Predicting, and calculating error"""   #approved
def predict_nn(shallow_nn, x, y):
    pred = shallow_nn.predict(x)
    y_pred = np.round(pred)
    y_pred = np.ravel(y_pred)
    valid_err = errcalc(y,y_pred)
    return y_pred, valid_err

In [10]:
"""Function for calculating Kappa"""
def kappa_calc(y, y_pred):
    
    if len(y) != len(y_pred):
        print('Input array sizes must match')
        return -1
    else:    
        #parameters
        total_data = len(y)
        agree_num = len(np.where(y == y_pred)[0])    #where model and specialists agree

        #specialists saying yes and no
        spec_yes = len(np.where(np.array(y) == 1)[0])
        spec_no = total_data - spec_yes

        #model saying yes
        model_yes = len(np.where(np.array(y_pred) == 1)[0])
        model_no = total_data - model_yes

        #calculating p_o
        p_o = agree_num / total_data

        #calculate p_e [raters randomly agreeing]
        rand_yes = (spec_yes/ total_data) * (model_yes/ total_data)
        rand_no = (spec_no / total_data) * (model_no / total_data)
        p_e = rand_yes + rand_no


        #kappa
        kappa = (p_o - p_e) / (1 - p_e)
        #kappa = 0
        print('kappa value is = ', kappa)
        print('po value is', p_o)
        print('agree num', agree_num)
        print('pe value is', p_e)
        print('rand_yes ', rand_yes)
        print('rand_no ', rand_no)
        print('spec_yes ' , spec_yes)
        print('spec_no', spec_no)
        print('model_yes ', model_yes)
        print('model_no ', model_no)
        return kappa

In [11]:
"""Function for calculating weighted error Epsilon""" #approved
def weighted_error(y,ypred,d):
    t = len(d)
    vec = np.abs(np.subtract(y,ypred))
    sumerror = 0.0
    for i in range (t):
        sumerror = sumerror + (d[i]*vec[i])
    return sumerror

In [12]:
def distr_upd(D, epsilon, alpha, y, h_x):       #approved
    z = 2.0* math.sqrt(epsilon*(1-epsilon))
    D_out = np.zeros(len(D))
    ##function to find mismatch vector
    ylabel = np.array(y)
    hypo = np.array(h_x)
    mismatch_vec = np.where(ylabel!=hypo, -1, 1)
    for i in range (len(D)):
        D_out[i] = (D[i]/z) * math.exp( (-alpha)* mismatch_vec[i])
    D_out = D_out / np.sum(D_out)
    return D_out/np.sum(D_out)

In [13]:
"""Function for Predicting Ensambled output"""   #approved
def ens_pred_nn(shallow_nn, x, y, alpha):
        nbsample, __ = np.shape(x)
        y_pred = np.zeros((nbsample))
        for nn, alph in zip(shallow_nn, alpha):
            y_temp, __ = predict_nn(nn, x, y)
            #added
            y_temp = np.array(y_temp)
            y_temp = np.where(y_temp==0, -1, 1)
            y_pred = y_pred + (alph * y_temp)
        
        y_pred = quantizer(y_pred)
        return y_pred  

In [14]:
"""Function for Sampling
Takes a 1-by-N matrix and outputs result as a one-dimensional matrix"""   #approved

def sampling(distr, n_samples):
    
    indvec = np.array(range(n_samples))
    out_ind=[]
    
    for i in range (n_samples):
        num = np.ceil(n_samples * distr[i])
        num = int(num)
        for j in range(num):
            out_ind.append(indvec[i])
    np.random.shuffle(out_ind)
    return out_ind[0:n_samples]

In [15]:
"""Function for quantizing predicted class values by custom AdaBoost implementation   
(so that classes take either 0 or 1 values)"""             #approved
def quantizer(y):
    yout=[]
    for i in range (len(y)):
        if(y[i]<0.0):
            yout.append(0.0)
        else:
            yout.append(1.0)
    yout = np.array(yout)
    return yout

In [16]:
"""Main program body"""
"""Sampling based ensamble"""

###Step 1: Load Training Data
#Done, not in a function.



### Adaboost flow
##=================================

# x,y = traindata, trainlabel
x,y = xtr, ytr

# initial = D = 1/n
nbsample = len(ytr)
D = (1/nbsample)* np.ones((nbsample,))

# variable to hold neural nets
nn_cluster = []

#variable to hold alpha values
alpha = []

for t in range (5):
    #1. create classifier
    shallow_nn = make_nn(10000, 8000, 7000, 6000, 5000, 4000, 2500, 1250, 650, 300, 100, 1)

    #2. Train using xtr
    train_nn(shallow_nn,x,y, 30, 50, True)

    #3. Predict on xtr
    ypred, __ = predict_nn(shallow_nn, x, y)

    #4. calculate epsilon
    epsilon = weighted_error(y,ypred,D)

    #5. Calculate alpha
    alph = 0.5*math.log10((1-epsilon)/epsilon)

    #6. update D
    D = distr_upd(D, epsilon, alph, y, ypred)

    #7. append the already made classifier
    nn_cluster.append(shallow_nn)

    #8. append the alpha values captured
    alpha.append(alph)

    #9. Sample data for next iteration
    sampled_ind = sampling(D,nbsample)
    x , y = x[sampled_ind,:], np.array(y)[sampled_ind]


### By this point, the classifier should be prepared, now we apply the predict function

ytr_predict = ens_pred_nn(nn_cluster, xtr, ytr, alpha)

### Calculating Error
train_error = errcalc(ytr, ytr_predict)

### Calculating Kappa

train_kappa = kappa_calc(ytr, ytr_predict)


## Printing Results

print('**************==============******************')
print('Training error is ', train_error, ' and kappa is ', train_kappa)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/40
1260/1260 [==============================] - 176s 140ms/step - loss: 0.7063
Epoch 2/40
1260/1260 [==============================] - 162s 128ms/step - loss: 0.6906
Epoch 3/40
1260/1260 [==============================] - 165s 131ms/step - loss: 0.6809
Epoch 4/40
1260/1260 [==============================] - 162s 128ms/step - loss: 0.6854
Epoch 5/40
1260/1260 [==============================] - 162s 128ms/step - loss: 0.7131
Epoch 6/40
1260/1260 [==============================] - 162s 129ms/step - loss: 0.6979
Epoch 7/40
1260/1260 [==============================] - 162s 128ms/step - loss: 0.6714
Epoch 8/40
1260/1260 [==============================] - 162s 128ms/step - loss: 0.6498
Epoch 9/40
1260/1260 [==============================] - 162s 129ms/step - loss: 0.6437
Epoch 10/40
1260/1260 [==============================] - 164s 130ms/step - loss: 0.6560
Epoch 11/

1260/1260 [==============================] - 162s 129ms/step - loss: 0.6772
Epoch 8/40
1260/1260 [==============================] - 162s 128ms/step - loss: 0.6556
Epoch 9/40
1260/1260 [==============================] - 162s 129ms/step - loss: 0.6909
Epoch 10/40
1260/1260 [==============================] - 162s 128ms/step - loss: 0.6972
Epoch 11/40
1260/1260 [==============================] - 163s 129ms/step - loss: 0.6753
Epoch 12/40
1260/1260 [==============================] - 164s 130ms/step - loss: 0.6660
Epoch 13/40
1260/1260 [==============================] - 163s 129ms/step - loss: 0.6386
Epoch 14/40
1260/1260 [==============================] - 161s 128ms/step - loss: 0.6389
Epoch 15/40
1260/1260 [==============================] - 162s 129ms/step - loss: 0.6451
Epoch 16/40
1260/1260 [==============================] - 162s 128ms/step - loss: 0.6477
Epoch 17/40
1260/1260 [==============================] - 162s 128ms/step - loss: 0.6636
Epoch 18/40
1260/1260 [=======================

1260/1260 [==============================] - 86s 68ms/step - loss: 0.5943
Epoch 21/40
1260/1260 [==============================] - 86s 68ms/step - loss: 0.6151
Epoch 22/40
1260/1260 [==============================] - 85s 68ms/step - loss: 0.5987
Epoch 23/40
1260/1260 [==============================] - 86s 68ms/step - loss: 0.7790
Epoch 24/40
1260/1260 [==============================] - 87s 69ms/step - loss: 0.5964
Epoch 25/40
1260/1260 [==============================] - 87s 69ms/step - loss: 0.5679
Epoch 26/40
1260/1260 [==============================] - 86s 68ms/step - loss: 0.5648
Epoch 27/40
1260/1260 [==============================] - 85s 68ms/step - loss: 0.6343
Epoch 28/40
1260/1260 [==============================] - 85s 68ms/step - loss: 0.5786
Epoch 29/40
1260/1260 [==============================] - 85s 68ms/step - loss: 0.5692
Epoch 30/40
1260/1260 [==============================] - 85s 68ms/step - loss: 0.7045
Epoch 31/40
1260/1260 [==============================] - 85s 68ms/

<h1>Loading Testing Data</h1>

In [17]:
"""Load Validation Data"""
#code for reading data
imgdir = []   #shall contain the full path of the image
yts = []         #shall contain label of each image
#xtr = np.matrix([[]])         #shall contain each image

#xtr = np.zeros((257,262144))   #modified
xts = np.zeros((1300,10000))

#datatype = ['train', 'valid']
#studylabel = {'negative':0, 'positive':1}

BASE_DIR = 'mura_humerus'

validdir = BASE_DIR + '/' + datatype[1]  #shows training directory

patients = os.listdir(validdir)  #all the patient ID's are now into this

i = 0
for patient in patients:
    directory = validdir + '/' + patient    #patient directory train/patientxxxx/
    studies = os.listdir(directory)         #creates list of studies in each patient directory
    for study in studies:
        images = os.listdir(directory + '/' + study)
        for image in images:
            imgdir = np.append(imgdir, directory + '/' + study + '/' + image)    #add image addresses in array
            yts = np.append(yts, studylabel[study.split('_')[1]])        #keep adding labels of each images
            file = mpimg.imread(imgdir[i])
            if (len(np.shape(file))>2):
                pixels = np.matrix(file[:,:,0])     #takes matrix format of one image
                pixels = np.reshape(pixels, (1,len(pixels)*len(pixels.T)))   #flattens the image to 1-D
            else:
                pixels = np.matrix(file)
                pixels = np.reshape(pixels, (1,len(pixels)*len(pixels.T)))
                
            if (np.shape(pixels)[1] < 10000):    #modified from 262144
                zeroarray = np.matrix(np.zeros((1,(10000-np.shape(pixels)[1]))))
                pixels = np.concatenate((pixels,zeroarray), axis=1)
                #print(pixels)
                #xtr = np.append(xtr, np.array(file[:,:,0]), axis=0)
                ###xtr = np.concatenate(xtr,pixels, axis=1)               #load and add images in 
                xts[i] = pixels               #load and add images in 
                #print ('i from if=', i)
            else:
                xts[i] = np.matrix(pixels[:,0:10000])     #modified from 262144
                #print(pixels)
                #print ('i from else=', i)
                
            i=i+1

#print(i)
xtest = xts[0:i,:]
#print(xtest.shape)
ytest = yts[0:i]
#print(i)
#print(np.shape(ytest))


"""Normalize data"""
for i in range (len(xtest)):
    if np.amax(xtest[i]) != 0:
        xtest[i] = xtest[i] / np.amax(xtest[i])
        
        
"""Eliminating all zero data"""
xts = np.zeros((np.shape(xtest)))
yts = []
j = 0
i = 0
while i < (len(ytest)):
    if np.amax(xtest[i]) != 0:
        xts[j] = xtest[i]
        yts.append(ytest[i])
        j = j+1
    i = i+1
    
xts = xts[0:j,:]




In [18]:
"""Verification"""
print(np.shape(xtest))
print(np.shape(xts))
print(np.shape(ytest))
print(np.shape(yts))

(288, 10000)
(282, 10000)
(288,)
(282,)


In [19]:
"""Evaluating test performance"""

### By this point, the classifier should be prepared, now we apply the predict function

yts_predict = ens_pred_nn(nn_cluster, xts, yts, alpha)

### Calculating Error
test_error = errcalc(yts, yts_predict)

### Calculating Kappa

test_kappa = kappa_calc(yts, yts_predict)


## Printing Results

print('**************==============******************')
print('Validation error is ', test_error, ' and kappa is ', test_kappa)


kappa value is =  0.11897203325774756
po value is 0.5602836879432624
agree num 158
pe value is 0.5009053870529652
rand_yes  0.21499170061868117
rand_no  0.285913686434284
spec_yes  139
spec_no 143
model_yes  123
model_no  159
**************==============******************
Validation error is  0.4397163120567376  and kappa is  0.11897203325774756
